In [24]:
import rasterio as rio
import numpy as np
import numpy.ma as ma
import xarray as xr
import os
from datetime import datetime, timedelta
from pyproj import Proj

#### Point to the directories with ABI data from each satellite and each band we want

In [44]:
filepaths = ['/home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C02/','/home/jovyan/goes17/2018/12/12/ABI-L1b-RadC/18/C02/',
             '/home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/','/home/jovyan/goes17/2018/12/12/ABI-L1b-RadC/18/C03/']
file_list = os.listdir(filepaths[1])

# listing the files for GOES-16, band 2 (red)
file_list

['OR_ABI-L1b-RadC-M3C02_G17_s20190131842190_e20190131844503_c20190131844528.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131802190_e20190131804562_c20190131804589.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131857190_e20190131859502_c20190131859527.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131822190_e20190131824562_c20190131824587.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131812190_e20190131814502_c20190131814526.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131852190_e20190131854562_c20190131854588.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131837190_e20190131839562_c20190131839587.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131832190_e20190131834562_c20190131834590.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131817190_e20190131819562_c20190131819588.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131807190_e20190131809562_c20190131809587.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131847190_e20190131849562_c20190131849589.nc',
 'OR_ABI-L1b-RadC-M3C02_G17_s20190131827190_e20190131829502_c20190131829527.nc']

#### Open NetCDF dataset using xarray

In [50]:
nc_file = filepaths[1] + file_list[3]
conus = xr.open_dataset(nc_file)

#### Look at the GOES Imager Projection information

In [51]:
conus.goes_imager_projection

<xarray.DataArray 'goes_imager_projection' ()>
array(-2147483647, dtype=int32)
Coordinates:
    t        datetime64[ns] ...
    y_image  float32 ...
    x_image  float32 ...
Attributes:
    long_name:                       GOES-R ABI fixed grid projection
    grid_mapping_name:               geostationary
    perspective_point_height:        35786023.0
    semi_major_axis:                 6378137.0
    semi_minor_axis:                 6356752.31414
    inverse_flattening:              298.2572221
    latitude_of_projection_origin:   0.0
    longitude_of_projection_origin:  -137.0
    sweep_angle_axis:                x

***
#### Function for generating a pyproj object (and proj4 string):

In [52]:
def getProj4string(goes_imager_projection):
    '''make a proj4 string  and a pyproj geostationary map object from goes imager projection information'''
    
    h=goes_imager_projection.perspective_point_height #          Satellite height above ellipsoid
    a=goes_imager_projection.semi_major_axis #                   Ellipsoid semi-major axis
    rf=goes_imager_projection.inverse_flattening #               Ellipsoid flattening (describes semi-minor axis)
    lon0=goes_imager_projection.longitude_of_projection_origin # Center longitude (GOES-East ~ -75.5)
    lat0=goes_imager_projection.latitude_of_projection_origin #  Center latitude (0.0)
    sweep=goes_imager_projection.sweep_angle_axis #              "Sweep angle" axis (x)
    
    # Make proj4 string
    proj_string = '"+proj=geos +ellps=GRS80 +h={h} +a={a} +rf={rf} +lon_0={lon0} +lat_0={lat0} +sweep={sweep}"'.format(
        h=h,a=a,rf=rf,lon0=lon0,lat0=lat0,sweep=sweep)
    
    # Use pyproj to define
    p = Proj(proj='geos', h=h, lon_0=lon0, sweep=sweep)
    
    return proj_string, p

In [53]:
# make a proj4 string and a pyproj geostationary map object from this file's projection information
proj_string, p = getProj4string(conus.goes_imager_projection)
print(proj_string)
print(p)

"+proj=geos +ellps=GRS80 +h=35786023.0 +a=6378137.0 +rf=298.2572221 +lon_0=-137.0 +lat_0=0.0 +sweep=x"
pyproj.Proj('+units=m +proj=geos +h=35786023.0 +lon_0=-137.0 +sweep=x ', preserve_units=True)


In [55]:
# use gdal_translate to convert to a GeoTiff including this projection information
tif_file = '{}.tif'.format(nc_file[:-3])
!gdal_translate -of GTiff -a_srs $proj_string NETCDF:$nc_file:Rad  $tif_file

Input file size is 10000, 6000
0...10...20...30...40...50...60...70...80...90...100 - done.
